XGBoost(Extreme Gradient Boosting) -2014
==========

1. XGBoost 기본 동작 방식
    -
   1)모든 데이터를 이용하여 모형을 학습한 후 적용한 데이터로부터 잔차(결과와의 차이)를 구한다.  
   2)잔차를 목적변수로 하여 동일한 특성변수 재적합.  
   3)1번 2번과정을 M번 반복. ->M번 반복 과정에서 overfitting 자주 발생  
     ->GBM과 동일한 알고리즘 사용
       회귀에 사용하는 경우 특성변수별 SSE를 구해서 평가.(XGBRegreesor 사용)
       분류에 사용하는 경우 infromation gain을 이용하여 평가.(XGBClassifier 사용)
2.XGBBoost의 장점
  -
  1) 과대적합을 방지하기 위한 정규화가 존재  
  2) 결측치 처리 방식을 지정하여 학습시킬 수 있음.  
  3) 임의의 특성변수를 일부만 뽑아 사용할 수도 있음.
 
3. 사용예시
  -

In [6]:
import pandas as pd
house =pd.read_csv('housing.data.txt.',header=None, sep='\s+')
house.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
house.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [8]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [14]:
X=house.iloc[:,:-1].values
y=house['MEDV'].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state =1)
xg_reg =xgb.XGBRegressor(objective ='reg:squarederror', booster='gbtree', colsample_bytree= 0.75, learning_rate = 0.1, max_depth=5, alpha = 10, n_estimators= 30)
xg_reg.fit(x_train, y_train)
pred_train = xg_reg.predict(x_train)
pred_test = xg_reg.predict(x_test)
rmse_train =np.sqrt(mean_squared_error(y_train, pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))
print('RMSE train %0.3f, test: %0.3f' %(rmse_train, rmse_test))

RMSE train 2.602, test: 3.725


함수인자 설명
===========
1.objectvie(손실함수)
 ---
->회귀: reg:squarederror가 default  
->이항분류: binary:losgistic  
->다항분류: multi:softmax or multi:softprob  

2.booster 추정에 사용할 모델
 ---
 
->gblinear 선형모형  
->gbtree 의사결정나무  
->dart 의사결정나무  

3.colsamplebytree(특성변수 사용량 결정 %단위)
 ---
->overfitting 방지

4.learning_rate(학습률)
 ---
5.max_depth(의사결정나무의 깊이 지정)
 ---
6.alpha (L1 regularization)
 ---
7.lambda(L2 regularization)
 ---
8.n_estimators(의사결정나무 사용갯수)
 ---

LightGBM -2017/1
==========

1.LightGBM의 장점
  -
  1)level_wise가 아닌 leaf-wise growth 이용  
   ![트리예시](Model_explain.jfif)  
 2)GOSS(Gradient-based One-Side Sampling) 사용   
   ->기울기가 큰 데이터 개체가 정보 획득에 있어 더욱 큰 역할을 한다는 아이디어에 입각해 만들어진 테크닉이다. 큰 기울기를 갖는 개체들은 유지되며, 작은 기울기  를 갖는 데이터 개체들은 일정 확률에 의해 랜덤하게 제거된다.  
 3)Exclusive feature building 사용ure building 사용
   희소행렬 사용시 column이 많아지는 결과가 나오게 되는데 이에 따른 훈련 시간이 오래걸리는 점이 있으므로 이를 위해 사용되는 차원축소 방식. 실수형 데이터에도    사용 가능.  
   **예시-> x1(0 부터 10), x2(0 부터 20)인 상태에서 만약 두 값이 동시에 0보다 큰 값을 가진 경우가 거의 없을 경우 0~30의 값을 가지는 새로운 칼럼 x3를 형성하는 방식으로 차원 축소** 

CatBoost -2017/4
==========

1. CatBoost가 가지는 차이점
   -
1)표본의 잔차를 보두 다른 모형으로부터 구함.  
2)범주형 데이터 변환으로 사용하는 one-hot label 대신 ordered target statistic 사용  
3)feature combination 자동 생성

| id | 시간 | x1 | y | y1 | y2
|:---:|:---:|:---:|:---:|:---:|:---|
|1|0|sunny|38|1|A|
|2|1|sunny|35|0|B|
|3|2|cloudy|20|1|C|
|4|3|cloudy|22|0|C|
|5|4|cloudy|21|1|B|
|6|5|sunny|33|1|A|
|7|6|cloudy|18|1|B|

1.잔차를 구하기 전 행해지는 것
   -  
1)random permutation에 의한 자료 뒤섞기
2)시간 column 자동 생성

2.ordered target statistic 적용방법
  -
  1)타겟값이 실수형인 경우
    ->해당 시간이전에 같은 레이블이 나타난 데이터의 타겟값의 합/같은 레이블의 수 (0/0 꼴 방지를 위해 분모에는 1을 더해주고 분자에는 target의 평균을 더해줌)
  2)타겟값이 이항분류인 경우
    ->실수형과 동일
  3)타겟값의 클래스가 여러개일 경우
    ->3개의 특성값을 가지는 feature로 변환